In [ ]:
from bs4 import BeautifulSoup as bs4
import requests
import pandas as pd
import re
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from webdriver_manager.chrome import ChromeDriverManager

from fake_useragent import UserAgent

import nltk
nltk.download("punkt")

pages = [
    "https://sol.sapo.pt/seccao/sociedade",
    "https://sol.sapo.pt/seccao/economia",
    "https://sol.sapo.pt/seccao/politica",
    "https://sol.sapo.pt/tag/ucrania",
    "https://sol.sapo.pt/seccao/internacional",
    "https://sol.sapo.pt/seccao/desporto",
    "https://sol.sapo.pt/seccao/tecnologia",
    "https://sol.sapo.pt/tag/covid",
]

In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Downloading: 100%|██████████| 6.30M/6.30M [00:01<00:00, 4.54MB/s]


In [3]:
# Open webpage to reject cookies and maximize window
driver.get("https://sol.sapo.pt/seccao/sociedade")

In [33]:
urls = set()
for page in pages:
    print("------------")
    print(page)
    print("------------")
    # Open webpage
    driver.get(page)
    
    time.sleep(3)

    # Scroll down until the end of the page
    lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    match=False
    while(match==False):
            lastCount = lenOfPage
            time.sleep(3)
            lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
            if lastCount==lenOfPage:
                match=True
    
    soup = bs4(driver.page_source, 'html.parser')
    links = soup.find_all("div", class_ = "content")
    for link in links:
        urls.add("https://sol.sapo.pt"+link.find("a")['href'])
        print("https://sol.sapo.pt"+link.find("a")['href'])

    quem_diria_section = soup.find("div", class_ = "large-3 column quemdiria-bar bggreen right")
    if quem_diria_section:
        quem_diria_links = quem_diria_section.find_all("div", class_ = "content")
        for link in quem_diria_links:
            urls.remove("https://sol.sapo.pt"+link.find("a")['href']) 
            print("quem diria", "https://sol.sapo.pt"+link.find("a")['href'])

------------
https://sol.sapo.pt/seccao/sociedade
------------
https://sol.sapo.pt/artigo/803450/pgr-da-resposta-automatica-a-montenegro
https://sol.sapo.pt/artigo/803444/sete-detidos-em-operacao-de-prevencao-criminal-na-guarda
https://sol.sapo.pt/artigo/803445/psp-recebeu-10-mil-queixas-desde-2019-relacionadas-com-abandono-e-maus-tratos-de-animais
https://sol.sapo.pt/artigo/803441/tripulantes-de-cabine-da-easyjet-em-greve-durante-cinco-dias
https://sol.sapo.pt/artigo/803439/portugal-teve-a-pior-campanha-de-cereais-de-sempre-em-2022
https://sol.sapo.pt/artigo/803437/homem-sequestra-companheira-em-beja-
https://sol.sapo.pt/artigo/803435/pj-realiza-operacao-contra-fraude-nos-acores
https://sol.sapo.pt/artigo/803432/mulher-apanhada-com-100-doses-de-heroina-em-visita-a-recluso
https://sol.sapo.pt/artigo/803428/interrompida-circulacao-na-linha-verde-do-metro-de-lisboa-
https://sol.sapo.pt/artigo/803421/ipma-coloca-vinte-e-cinco-concelhos-em-perigo-maximo-de-inc-ndio
https://sol.sapo.pt/arti

In [30]:
for url in urls:
    print(url)

https://sol.sapo.pt/artigo/802820/psp-deteve-seis-pessoas-e-apreendeu-mais-de-mil-doses-de-haxixe-no-faial
https://sol.sapo.pt/artigo/803279/guarda-civil-detem-dois-portugueses-e-apreende-mais-de-tr-s-quilos-de-canabis-em-badajoz
https://sol.sapo.pt/artigo/803274/atropelamento-que-matou-12-ovelhas-condicionou-circulacao-ferroviaria-na-linha-do-minho
https://sol.sapo.pt/artigo/803358/prisao-preventiva-para-suspeito-de-atear-fogo-numa-habitacao-em-sabrosa
https://sol.sapo.pt/artigo/803369/fisco-emite-novo-alerta-sobre-email-fraudulento-desta-vez-sobre-a-declaracao-de-irs
https://sol.sapo.pt/artigo/802788/homem-atropelado-na-a1-em-sacavem
https://sol.sapo.pt/artigo/803340/detido-homem-por-abusar-sexualmente-de-criancas-de-8-e-11-anos-em-vila-real
https://sol.sapo.pt/artigo/803110/prevista-a-supressao-de-105-comboios-na-estacao-de-campanha-
https://sol.sapo.pt/artigo/803085/homem-morre-em-atropelamento-ferroviario-em-portalegre
https://sol.sapo.pt/artigo/803050/psicoterapia-assistida-por-k

In [44]:
len(urls) #284

954

In [59]:
urls_test = [
    "https://sol.sapo.pt/artigo/800660/secretario-das-financas-chamado-de-urg-ncia-ao-parlamento-por-causa-de-certificados-de-aforro",
    "https://sol.sapo.pt/artigo/802632/r-ssia-abate-missil-ucraniano-na-regiao-fronteirica-de-rostov",
    "https://sol.sapo.pt/artigo/799433/teia-de-gaia-apanha-governantes",
]

In [60]:
texts=[]
sources=[]
df_urls = []
# Initialize an empty dataframe
df = pd.DataFrame(columns=['Text', 'Source', 'URL'])

for url in urls:
    # if url.split("https://expresso.pt")[1] == "":
    #     url = "https://expresso.pt" + url.split("https://expresso.pt")[2]
    print("-------------")
    print(url)
    print("-------------")
    text = "NONE"
    source = "SOL"
    #exclusive_article = False

    time.sleep(4)

    try:
        driver.get(url)

        1
        soup = bs4(driver.page_source, 'html.parser')

        article_elem = soup.find("div", class_ = "large-9 medium-8 column artic_content")
        header = article_elem.find("header")
        title_1 = header.find("h1").get_text().strip()
        title_2_elem = header.find_all("p")
        for elem in title_2_elem:
            if elem.get_text().strip():
                title_2= elem.get_text().strip()
        #print(title_1)
        #print(title_2)
        body_elem = article_elem.find("div", class_ = "large-8 column corpo")
        paragraphs = body_elem.find_all("p")
        #print(paragraphs)
        news_text = ""
        for paragraph in paragraphs:
            #print(paragraph.get_text().strip())
            news_text += "\n" + paragraph.get_text().strip()
        
        text = title_1 + "\n" + title_2 + "\n" + news_text
        text = re.sub(r'\n+', '\n', text) # Replace consecutive paragraphs with a single one
        text = text.strip()

    except:
        pass
    
    texts.append(text)
    sources.append(source)
    df_urls.append(url)

    print(text)
    print(source)

    time.sleep(3)
        
# Create a new dataframe from the lists
new_data = pd.DataFrame({'Text': texts, 'Source': sources, 'URL': df_urls})

# Concatenate the new dataframe with the existing dataframe
df = pd.concat([df, new_data], ignore_index=True)

-------------
https://sol.sapo.pt/artigo/802632/r-ssia-abate-missil-ucraniano-na-regiao-fronteirica-de-rostov
-------------
Rússia abate míssil ucraniano na região fronteiriça de Rostov
Foram danificados, parcialmente, os telhados de vários edifícios.
Um míssil ucraniano foi, este domingo, abatido pelas defesas antiaéreas russas na região russa de Rostov, na fronteira com a Ucrânia, disse o governador daquela província, Vasili Gólubev.
O ataque, diz a mesma fonte, aconteceu pelas 12h00 (9h00, em Lisboa). “O nosso sistema de defesa aéreo funcionou rapidamente, escreveu Góluven, através do Telegram, acrescentando que não foram registadas vítimas.
Ainda assim, foram danificados, parcialmente, os telhados de vários edifícios.
SOL
-------------
https://sol.sapo.pt/artigo/800660/secretario-das-financas-chamado-de-urg-ncia-ao-parlamento-por-causa-de-certificados-de-aforro
-------------
Secretário das Finanças chamado de "urgência" ao Parlamento por causa de Certificados de Aforro
Nova série d

In [61]:
len(urls)

954

In [62]:
len(texts)

954

In [63]:
len(sources)

954

In [64]:
len(df_urls)

954

In [65]:
for url in df_urls:
    print(url)

https://sol.sapo.pt/artigo/802632/r-ssia-abate-missil-ucraniano-na-regiao-fronteirica-de-rostov
https://sol.sapo.pt/artigo/800660/secretario-das-financas-chamado-de-urg-ncia-ao-parlamento-por-causa-de-certificados-de-aforro
https://sol.sapo.pt/artigo/765529/ex-trabalhador-da-apple-acusado-de-defraudar-a-empresa-em-nove-milhoes-de-euros
https://sol.sapo.pt/artigo/802518/secretario-de-estado-da-defesa-nacional-pede-demissao
https://sol.sapo.pt/artigo/800002/campeonato-ltimo-dia-da-criacao
https://sol.sapo.pt/artigo/803110/prevista-a-supressao-de-105-comboios-na-estacao-de-campanha-
https://sol.sapo.pt/artigo/803443/r-ssia-lancou-novo-ataque-contra-infraestruturas-em-odessa-e-zaporijia
https://sol.sapo.pt/artigo/784723/microsoft-ira-continuar-a-apoiar-ucr-nia-com-mais-de-100-milhoes-de-euros
https://sol.sapo.pt/artigo/799953/grupo-credito-agricola-com-lucros-de-95-8-milhoes-no-primeiro-trimestre
https://sol.sapo.pt/artigo/801240/quando-pedro-nuno-santos-quiser-voltar-a-primeira-linha-vai-

In [66]:
df.head()

,Text,Source,URL
0,Rússia abate míssil ucraniano na região fronte...,SOL,https://sol.sapo.pt/artigo/802632/r-ssia-abate...
1,"Secretário das Finanças chamado de ""urgência"" ...",SOL,https://sol.sapo.pt/artigo/800660/secretario-d...
2,Ex-trabalhador da Apple acusado de defraudar a...,SOL,https://sol.sapo.pt/artigo/765529/ex-trabalhad...
3,Secretário de Estado da Defesa Nacional pede d...,SOL,https://sol.sapo.pt/artigo/802518/secretario-d...
4,"Campeonato. Último dia da criação\nHoje, pelas...",SOL,https://sol.sapo.pt/artigo/800002/campeonato-l...


In [67]:
df.head()
df.to_csv("Jornal_SOL_dataset.csv", index=False)

In [68]:
loaded_df = pd.read_csv("Jornal_SOL_dataset.csv")
loaded_df.head()

,Text,Source,URL
0,Rússia abate míssil ucraniano na região fronte...,SOL,https://sol.sapo.pt/artigo/802632/r-ssia-abate...
1,"Secretário das Finanças chamado de ""urgência"" ...",SOL,https://sol.sapo.pt/artigo/800660/secretario-d...
2,Ex-trabalhador da Apple acusado de defraudar a...,SOL,https://sol.sapo.pt/artigo/765529/ex-trabalhad...
3,Secretário de Estado da Defesa Nacional pede d...,SOL,https://sol.sapo.pt/artigo/802518/secretario-d...
4,"Campeonato. Último dia da criação\nHoje, pelas...",SOL,https://sol.sapo.pt/artigo/800002/campeonato-l...


In [69]:
for source in set(loaded_df['Source']):
    print(source)

SOL


In [70]:
num_rows_with_none = len(loaded_df[loaded_df['Text'] == 'NONE'])
print(num_rows_with_none)

4


In [71]:
len(loaded_df)

954

In [72]:
final_dataset = df[df["Text"] != "NONE"]

In [73]:
len(final_dataset)

950

In [74]:
final_dataset.head()
final_dataset.to_csv("Jornal_SOL_dataset.csv", index=False)